Export simple multilayer perceptron to mcfunction (math api required)

In [1]:
import numpy as np
import torch

Input: several values; Output: 1 value

In [2]:
def fnn_func_autoscale(fname,sd,lastrelu=False,vmax=1000000): # vmax is max value likely encountered.
    '''
    Convert a model to mcfunction.
    Parameters are scaled by 10000 for int32 scoreboard operations.
    So your input parameter also need to be scaled.
    For example, horizontal distance of 10m should be scaled to 100000 before calling the function.
    vmax should be the maximum input value the network encounter to prevent overflows.
    '''
    
    mtp = int(np.floor(2**31/vmax))
    f = open(fname,'w')

    f.write('\n\n\n\n')

    ks = list(sd.keys())
    ll = int(len(ks)/2)

    for l in range(ll):
        f.write(f'#Process Layer {l}\n\n')
        wl = np.int32(np.round(sd[ks[int(l*2)]]*10000))
        bl = np.int32(np.round(sd[ks[int(l*2+1)]]*10000))
        #print(np.max(np.abs(wl)),np.max(np.abs(bl)))
        maxw = np.max(np.abs(wl))
        #print('L----')
        #print(maxw)
        div0 = int(np.ceil(maxw/mtp))
        div1 = int(round(10000/div0))
        #print(div0,div1)

        f.write(f'scoreboard players set #div0 swMath_V {div0}\n')
        f.write(f'scoreboard players set #div1 swMath_V {div1}\n')
        # scale
        for j in range(wl.shape[1]):
            f.write(f'scoreboard players operation #x{l}_{j} swMath_V /= #div0 swMath_V\n')
        f.write('\n')
        for i in range(wl.shape[0]):

            f.write(f'scoreboard players set #x{l+1}_{i} swMath_V 0\n')
            f.write(f'scoreboard players set #bl_{i} swMath_V {bl[i]}\n')

            for j in range(wl.shape[1]):
                f.write(f'scoreboard players set #wl_{i}{j} swMath_V {wl[i][j]}\n')
                f.write(f'scoreboard players operation #wl_{i}{j} swMath_V *= #x{l}_{j} swMath_V\n')
                f.write(f'scoreboard players operation #wl_{i}{j} swMath_V /= #div1 swMath_V\n')
                f.write(f'scoreboard players operation #x{l+1}_{i} swMath_V += #wl_{i}{j} swMath_V\n')

            f.write(f'scoreboard players operation #x{l+1}_{i} swMath_V += #bl_{i} swMath_V\n')
            if l+1 != ll:
                f.write(f'execute if score #x{l+1}_{i} swMath_V matches ..-1 run scoreboard players set #x{l+1}_{i} swMath_V 0\n')
            elif lastrelu:
                f.write(f'execute if score #x{l+1}_{i} swMath_V matches ..-1 run scoreboard players set #x{l+1}_{i} swMath_V 0\n')
            #f.write(f'tellraw @a [{{"text":"#x1_{i}: "}},{{"score":{{"name": "#x1_{i}","objective": "swMath_V"}}}}]')

            f.write('\n\n')
        f.write('\n\n')
    f.write(f'scoreboard players operation #x_out swMath_V = #x{l+1}_{i} swMath_V \n')
    f.close()
    pass

In [24]:
net = torch.jit.load('model/xxx_a').to('cpu')
sd = {}
for e in net.state_dict():
    sd[e]=net.state_dict()[e].cpu().numpy()

fname = 'path_to_file.mcfunction'
fnn_func_autoscale(fname,sd,lastrelu=False)

In [ ]:
net = torch.jit.load('model/xxx_t').to('cpu')
sd = {}
for e in net.state_dict():
    sd[e]=net.state_dict()[e].cpu().numpy()

fname = 'path_to_file.mcfunction'
fnn_func_autoscale(fname,sd,lastrelu=True)